In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

print(np.__version__)
print(tf.__version__)
print(cv2.__version__)

1.18.5
2.1.3
4.5.1


In [ ]:
## The fully convolutional encoder-decoder network and then creating a new loss function with the VGG_19 loss
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf


vgg_19 = tf.keras.applications.VGG19(include_top=False,weights='imagenet',input_shape=(512,512,3),pooling=None)
vgg_19.summary()

new_vgg = Model(inputs= vgg_19.input,outputs=vgg_19.get_layer('block5_conv4').output)
new_vgg.summary()

In [ ]:
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras as keras
## The fully convolutional encoder-decoder network and then creating a new loss function with the VGG_19 loss
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf



def build_model():
    input_img = Input(shape=(256, 256, 3))
    l1 = Conv2D(64, (3, 3), padding='same', activation='relu', bias_initializer='zeros',
                activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(input_img)
    l2 = Conv2D(64, (3, 3), padding='same', activation='relu', bias_initializer='zeros',
                activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l1)
    print('This is the l2\'s shape {}'.format(l2.shape))

    l3 = MaxPooling2D(padding='same')(l2)
    print(l3.shape)
    #l3 = BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001)(l3)
    l3 = Dropout(0.3)(l3)
    l4 = Conv2D(128, (3, 3),  padding='same', activation='relu', bias_initializer='zeros',
                activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l3)
    l5 = Conv2D(128, (3, 3), padding='same', activation='relu', bias_initializer='zeros',
                activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l4)
    print('This is L5 shape {}'.format(l5.shape))

    l6 = MaxPooling2D(padding='same')(l5)
    l6 = Dropout(0.3)(l6)
    #l6 = BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001)(l6)
    l7 = Conv2D(256, (3, 3), padding='same', activation='relu', bias_initializer='zeros',
                activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l6)
    print('This is L7 shape {}'.format(l7.shape))
    #l8 = UpSampling2D()(l7)
    l8 = Conv2DTranspose(128,(1,1),padding='same',strides=(2,2),activation='relu',bias_initializer='zeros',activity_regularizer=regularizers.l1(10e-10),
                        kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32),data_format='channels_last')(l7)
    # increasing the stide to 2,2 enables upsampling of the images
    print('This is L8 shape {}'.format(l8.shape))
    #l8 = BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001)(l8)
    l8 = Dropout(0.3)(l8)
    l9 = Conv2D(128, (3, 3), padding='same', activation='relu',bias_initializer='zeros',
                activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l8)
    l10 = Conv2D(128, (3, 3), padding='same', activation='relu',bias_initializer='zeros',
                 activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l9)
    #l10 = BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001)(l10)
    l10 = Dropout(0.3)(l10)

    l11 = add([l5, l10])
    
    print('This is the l11 shape {}'.format(l11.shape))
    #l12 = UpSampling2D()(l11)
    l12 = Conv2DTranspose(128,(2,2),padding='same',strides=(2,2),activation='relu',bias_initializer='zeros',activity_regularizer=regularizers.l1(10e-10),
                        kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32),data_format='channels_last')(l11)
    
    print('This is the l12\'s shape {}'.format(l12.shape))
    
    l13 = Conv2D(64, (3, 3), padding='same', activation='relu',bias_initializer='zeros',
                 activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l12)
    l14 = Conv2D(64, (3, 3), padding='same', activation='relu',
                 activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l13)
    
    #l14 = BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001)(l14)
    l14 = Dropout(0.3)(l14)
    l15 = add([l14, l2])
    l16 = Conv2DTranspose(64,(2,2),padding='same',strides=(2,2),activation='relu',bias_initializer='zeros',activity_regularizer=regularizers.l1(10e-10),
                        kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32),data_format='channels_last')(l15)

    decoded = Conv2D(3, (3, 3), padding='same', activation='relu', bias_initializer='zeros',
                     activity_regularizer=regularizers.l1(10e-10),kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(seed=None,dtype=tf.dtypes.float32))(l16)

    print('This is the decoder\'s shape {}'.format(decoded.shape))
    model = Model(input_img, decoded)
    #model.compile(optimizer=keras.optimizers.Adam(1e-4), loss='mean_squared_error')
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(optimizer=keras.optimizers.Adam(1e-4), loss=vgg_19_loss)
    return model